# WALL mass swaps

WALL had a campaign to incentivize token swaps with 1TON value (https://t.me/wall_jetton/156).
Some users made chains of wallets, with such pattern:
1. Buy WALL
2. Send all the remaining balance to a new wallet
3. ...
4. PROFIT!

Initial funds came from xRocket. To get the list of all starting wallets the following Tonalytica query 
was used (finds all wallets with only one swaps TON/WALL and incoming message from xRocket):

```sql
select distinct swap_user from redoubt.dex_swaps dsp where 
swap_dst_token='EQDdCha_K-Z97lKl599O0GDAt0py2ZUuons4Wuf85tq6NXIO'
and swap_time  > '2024-08-01'::date
and exists (select from redoubt.messages where 
  source = 'EQABGo8KCza3ea8DNHMnSWZmbRzW-05332eTdfvW-XDQEjQM'
  and destination =swap_user
  and value > 50 * 1e9
)
and (select count(1) from redoubt.dex_swaps dsp2 where dsp2.swap_user  = dsp.swap_user) = 1

```

Total: **1476** wallets

In [80]:
start = [
    # dedust
    'EQA1itS1KXSm5jU4CBL1Bipv8mRmVfGpnEtEfDOZ8jWndX3d',
    'EQAVMBf6uxVZ2ZD8PEuHYRo5Wmb6vJNHhi6kXAyJoM_Ncyxy',
    'EQBcwVVe7g9GTx-w1o4XUdYFalQuGw9s-RbyybPu4CPfidz2',
    'EQBewfQ1Aeem3U0Qf5_i8sv27bVriuPnBX9H_luZLP1ntTb3',
    'EQBJceWxGZXtxvgBXJUd8d8FUQFlcFoJpHsNXH365GBjeYQT',
    'EQBkoRmsmIzOpfkvGT8XCVswTWJm1OaatOADVwHpAP-TVVvG',
    'EQBNdLXUL0-tgA3EpLNIjT87Th-CkF_iVuTkxLfw3tIW3bTy',
    'EQBtWTnRVMSp-HqlExsDwcV3SXKdTUeT47ntHms5KWbKDyfq',
    'EQBz8Etqj5ps1A7s5W4o4_taQPLh5PG7sM0MbysmJU5DvOVI',
    'EQCAIbL5545-UMveKHJfrz8mPgEwxk-xXl0XydKqJ5YfWwk_',
    'EQCb8qFbXWIjvJsoHZxyR7hzcpY3tqgm7JdBmezOvWOIBiN_',
    'EQCk1LAssFEGnB7hhFEOPLGtemry3yBDSZtPTtIRRNkkMteL',
    'EQDFFVSPIw038qVppOBRValaP6Ei_p242n-NbO5bgacIvn6q',
    'EQDVq6t4fJ7itbi6HjzX9LrZwciqEMos0PTYfs4A0rYBztrC',
    'EQDwfvndRDGQibbnpSy1b3Pcqoznd1t8GNGF3tiyHTySDnBL',
    
    # ston.fi
    'EQAW64hDavau7m9p3TRWkCVLMIp5TiJ6cBKMlx7UcjPhWrfs',
    'EQAwi6il0MfNFkgEE78MmToHDs_eTqwa5NQqPvObGcAZFmlo',
    'EQB9BZx7J0VVveqS1dlm2wkiS-jSzgGsG7grQV4ARJa3DnpE',
    'EQDdxSU8aqiCELYFWzDI7lEQAworBVNvA0inhwqMjE5U_nlp',
    'EQDmmHdqI5e_dzt0xK3FX6etDtijouY3GVrM4pB26YRr7qUe',
    
]
#sybils = set()

In [ ]:
import requests 
import time
from tonsdk.utils import Address
from tqdm import tqdm
DEDUST_VAULT = '0:DAE153A74D894BBC32748198CD626E4F5DF4A69AD2FA56CE80FC2644B5708D20'
STONFI_VAULT = '0:1150B518B2626AD51899F98887F8824B70065456455F7FE2813F012699A4061F'

to_check = start
with tqdm() as pbar:
    while True:
        if len(to_check) == 0:
            break

        next_round = set()
        for address in to_check:
            sybils.add(address)
            pbar.update(1)
            res = requests.get(f"https://toncenter.com/api/v3/messages?source={address}", headers={}).json()
            assert res.get('ok', True)
            has_swap = False
            for m in res['messages']:
                if m['destination'] == DEDUST_VAULT or m['destination'] == STONFI_VAULT:
                    has_swap = True
            if has_swap:
                for m in res['messages']:
                    if m['destination'] != DEDUST_VAULT and m['destination'] != STONFI_VAULT:
                        new_addr = Address(m['destination']).to_string(1, 1, 1)
                        next_round.add(new_addr)


    #         time.sleep(2)
        to_check = next_round

In [76]:
import pandas as pd
df = pd.DataFrame(sybils, columns=['address']).query("address != ''")
df['project'] = 'WALL'
df['reason'] = 'mass-accounts'
df.to_csv("wall_banned.csv", index=False)

In [79]:
from IPython.core.display import display, HTML

with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.width', None):
    display(HTML(df.to_html()))

,address,project,reason
1,EQDdDs1y8SZKC0DQ9olbf7Dj8YTQ_3djeKh02SfI6W3f4-JC,WALL,mass-accounts
2,EQAm8P1PW2Dm6SE73pBUlgVZCIZZBJv4JNe21mvohgbA1QYi,WALL,mass-accounts
3,EQDiGuPlu_MzrzBCBXRxIM1psQgZ3E3OpO3M5YPWBgXApalS,WALL,mass-accounts
4,EQCVaQAND0sYeposdHthiQ18ErwX2EWHMekoxI5IroQ3AQOk,WALL,mass-accounts
5,EQC_Tr8mhzDNq2P4rAbt3qAJKd__jlM__8VW_pOlhswLJvpc,WALL,mass-accounts
6,EQCCfyFv9VuOX8IMKZ3tjDUyNt0XelC7lCJcKeQ27y_66SFi,WALL,mass-accounts
7,EQBewfQ1Aeem3U0Qf5_i8sv27bVriuPnBX9H_luZLP1ntTb3,WALL,mass-accounts
8,EQBmrbl9ZXJUPQaImGc1Y3eXAlRf5Wp0kDNmNM0dFcxhCjHN,WALL,mass-accounts
9,EQDg5gOmAg2JPKKsZJjkdUIifswZWAc5fyQCMq_d3Tklh7Ck,WALL,mass-accounts
10,EQBa8wi42oSgh20vvYssy5P4LQNK6lllYI089KmI6RaO_CdZ,WALL,mass-accounts
